In [3]:
import importlib

import os 
from nltk.corpus import movie_reviews
import random 

import classifier
# from classifier import train_naive_bayes
import utils 
import evaluation

importlib.reload(classifier)
importlib.reload(evaluation)

<module 'evaluation' from 'c:\\Users\\adamk\\NLP_Project\\NLP_SemanticAnalysis\\evaluation.py'>

## Load Data

Below is a helper function to read in file and separate the words into a list

In [6]:
def read_in_txt(fname):
  with open(fname) as in_file:
    in_file_str = in_file.read()
    file_contents = in_file_str.split("\n")

  return file_contents

### Training Data

In [7]:
neg_words = read_in_txt("movie_reviews/negWords.txt")
pos_words = read_in_txt("movie_reviews/posWords.txt")

print("num negative words:", len(neg_words))
print("num positive words:", len(pos_words))

num negative words: 296450
num positive words: 333400


### Testing Data

In [10]:

pos_test_files = read_in_txt("movie_reviews/posTst.txt")
neg_test_files = read_in_txt("movie_reviews/negTst.txt")

pos_test = [movie_reviews.words(fname) for fname in pos_test_files]
neg_test = [movie_reviews.words(fname) for fname in neg_test_files]

test_words = pos_test + neg_test

y_test = [1 for i in range(len(pos_test))] + [0 for i in range(len(neg_test))]


## Training
Train the model using the bags of positive and negative words

In [11]:
priors, likelihoods = classifier.train_naive_bayes(pos_words, neg_words)

## Testing 
Using the test phrases and the results from the training function above generate predictions.

In [12]:
predicted = classifier.predict_naive_bayes(priors, likelihoods, test_words)

## Evalution of Results
The confusion matrix below displays the results from our classifier

In [13]:
matrix = evaluation.confusion_matrix(y_test, predicted, [0, 1])

positives = matrix[0]
negatives = matrix[1]

print("True Positives:", positives[0])
print("False Positives:", positives[1])
print("False Negatives:", negatives[0])
print("True Negatives:", negatives[1])

True Positives: 93
False Positives: 7
False Negatives: 56
True Negatives: 44
